# Public Company Financial Data

Author: Phuoc Le<br>
Google Colab Version: Version 2<br>
Last Edited: September 25, 2020<br>
Data Source: Yahoo Finance

## Libraries Import

Import Python libraries needed for the notebook.  First we need to install some additional libraries.

In [ ]:
#@title
!pip install yahoo_fin

In [ ]:
#@title
!pip install requests_html

In [40]:
import yahoo_fin.stock_info as si
import pandas as pd
import plotly.express as px
from tqdm import tqdm

## Define List of Companies to Research

Enter list of stocks as a Python list

In [62]:
companies = ["AAPL", "MSFT", "WMT", "TGT", "MCD", "KO", "KMB", "RY"]

## Retrieve Financial Information for our Companies

1. The first thing to do is to retrieve the data.  Depending on how long the company list is, it may take a while.  We can't run Step 2 to clean up the data until this is complete, so be patient!  Wait for the indicator to get to 100% before proceeding.

In [63]:
#@title
# Get data in the current column for each stock's valuation table
# df1 contains basic company data
# df2 contains extra data

ticker_stats = {}
for ticker in companies:
    temp = si.get_stats_valuation(ticker)
    temp = temp.iloc[:,:2]
    temp.columns = ["Attribute", "Recent"]
 
    ticker_stats[ticker] = temp


# combine all the stats valuation tables into a single data frame
df1 = pd.concat(ticker_stats)
df1 = df1.reset_index()

del df1["level_1"]

# update column names
df1.columns = ["Ticker", "Attribute", "Recent"]

# Get extra stats
ticker_extra_stats = {}
for ticker in tqdm(companies):
    ticker_extra_stats[ticker] = si.get_stats(ticker)
    

df2 = pd.concat(ticker_extra_stats)

df2 = df2.reset_index()

del df2["level_1"]

df2.columns = ["Ticker", "Attribute", "Value"]

100%|██████████| 8/8 [00:03<00:00,  2.19it/s]


2. Now we do the data clean up, type conversions, and merging of the data into a final dataframe called **company_data** which we can now use.

In [64]:
#@title
# Convert from long dataframe to wide dataframe using the pivot() method.
df1_wide = df1.pivot(index = "Ticker", columns="Attribute", values="Recent")
df2_wide = df2.pivot(index = "Ticker", columns="Attribute", values="Value")

# Clean up column names to remove footnote indicators
df1_wide.rename(columns = {"Enterprise Value 3":"Enterprise Value"}, inplace = True)
df1_wide.rename(columns = {"Enterprise Value/EBITDA 6":"Enterprise Value/EBITDA"}, inplace = True)
df1_wide.rename(columns = {"Enterprise Value/Revenue 3":"Enterprise Value/Revenue"}, inplace = True)
df1_wide.rename(columns = {"Forward P/E 1":"Forward P/E"}, inplace = True)
df1_wide.rename(columns = {"Market Cap (intraday) 5":"Market Cap (intraday)"}, inplace = True)
df1_wide.rename(columns = {"PEG Ratio (5 yr expected) 1":"PEG Ratio (5 yr expected)"}, inplace = True)

df2_wide.rename(columns = {"% Held by Insiders 1":"% Held by Insiders"}, inplace = True)
df2_wide.rename(columns = {"% Held by Institutions 1":"% Held by Institutions"}, inplace = True)
df2_wide.rename(columns = {"200-Day Moving Average 3":"200-Day Moving Average"}, inplace = True)
df2_wide.rename(columns = {"5 Year Average Dividend Yield 4":"5 Year Average Dividend Yield %"}, inplace = True)
df2_wide.rename(columns = {"50-Day Moving Average 3":"50-Day Moving Average"}, inplace = True)
df2_wide.rename(columns = {"52 Week High 3":"52 Week High"}, inplace = True)
df2_wide.rename(columns = {"52 Week Low 3":"52 Week Low"}, inplace = True)
df2_wide.rename(columns = {"52-Week Change 3":"52-Week Change %"}, inplace = True)
df2_wide.rename(columns = {"Avg Vol (10 day) 3":"Avg Vol (10 day)"}, inplace = True)
df2_wide.rename(columns = {"Avg Vol (3 month) 3":"Avg Vol (3 month)"}, inplace = True)
df2_wide.rename(columns = {"Dividend Date 3":"Dividend Date"}, inplace = True)
df2_wide.rename(columns = {"Ex-Dividend Date 4":"Ex-Dividend Date"}, inplace = True)
df2_wide.rename(columns = {"Forward Annual Dividend Rate 4":"Forward Annual Dividend Rate"}, inplace = True)
df2_wide.rename(columns = {"Forward Annual Dividend Yield 4":"Forward Annual Dividend Yield %"}, inplace = True)
df2_wide.rename(columns = {"Last Split Date 3":"Last Split Date"}, inplace = True)
df2_wide.rename(columns = {"Last Split Factor 2":"Last Split Factor"}, inplace = True)
df2_wide.rename(columns = {"Operating Margin (ttm)":"Operating Margin (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"Payout Ratio 4":"Payout Ratio %"}, inplace = True)
df2_wide.rename(columns = {"Profit Margin":"Profit Margin %"}, inplace = True)
df2_wide.rename(columns = {"Quarterly Earnings Growth (yoy)":"Quarterly Earnings Growth (yoy) %"}, inplace = True)
df2_wide.rename(columns = {"Quarterly Revenue Growth (yoy)":"Quarterly Revenue Growth (yoy) %"}, inplace = True)
df2_wide.rename(columns = {"Return on Assets (ttm)":"Return on Assets (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"Return on Equity (ttm)":"Return on Equity (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"S&P500 52-Week Change 3":"S&P500 52-Week Change %"}, inplace = True)
df2_wide.rename(columns = {"Shares Outstanding 5":"Shares Outstanding"}, inplace = True)
df2_wide.rename(columns = {"Trailing Annual Dividend Rate 3":"Trailing Annual Dividend Rate"}, inplace = True)
df2_wide.rename(columns = {"Trailing Annual Dividend Yield 3":"Trailing Annual Dividend Yield %"}, inplace = True)

# Convert string values to float as necessary
df1_wide['Trailing P/E'] = df1_wide['Trailing P/E'].astype(float)
df1_wide['Enterprise Value/EBITDA'] = df1_wide['Enterprise Value/EBITDA'].astype(float)
df1_wide['Enterprise Value/Revenue'] = df1_wide['Enterprise Value/Revenue'].astype(float)
df1_wide['Forward P/E'] = df1_wide['Forward P/E'].astype(float)
df1_wide['PEG Ratio (5 yr expected)'] = df1_wide['PEG Ratio (5 yr expected)'].astype(float)
df1_wide['Price/Book (mrq)'] = df1_wide['Price/Book (mrq)'].astype(float)
df1_wide['Price/Sales (ttm)'] = df1_wide['Price/Sales (ttm)'].astype(float)

df2_wide['% Held by Insiders'] = df2_wide['% Held by Insiders'].str.strip('%').astype('float') / 100.0
df2_wide['% Held by Institutions'] = df2_wide['% Held by Institutions'].str.strip('%').astype('float') / 100.0
df2_wide['5 Year Average Dividend Yield %'] = df2_wide['5 Year Average Dividend Yield %'].astype(float) / 100.0
df2_wide['50-Day Moving Average'] = df2_wide['50-Day Moving Average'].astype(float)
df2_wide['52 Week High'] = df2_wide['52 Week High'].astype(float)
df2_wide['52 Week Low'] = df2_wide['52 Week Low'].astype(float)
df2_wide['52-Week Change %'] = df2_wide['52-Week Change %'].str.strip('%').astype('float') / 100.0
df2_wide['Beta (5Y Monthly)'] = df2_wide['Beta (5Y Monthly)'].astype(float)
df2_wide['Book Value Per Share (mrq)'] = df2_wide['Book Value Per Share (mrq)'].astype(float)
df2_wide['Current Ratio (mrq)'] = df2_wide['Current Ratio (mrq)'].astype(float)
df2_wide['Diluted EPS (ttm)'] = df2_wide['Diluted EPS (ttm)'].astype(float)
df2_wide['Forward Annual Dividend Rate'] = df2_wide['Forward Annual Dividend Rate'].astype(float)
df2_wide['Forward Annual Dividend Yield %'] = df2_wide['Forward Annual Dividend Yield %'].str.strip('%').astype('float') / 100.0
df2_wide['Operating Margin (ttm) %'] = df2_wide['Operating Margin (ttm) %'].str.strip('%').astype('float') / 100.0
df2_wide['Payout Ratio %'] = df2_wide['Payout Ratio %'].str.strip('%').astype('float') / 100.0
df2_wide['Profit Margin %'] = df2_wide['Profit Margin %'].str.strip('%').astype('float') / 100.0
df2_wide['Quarterly Earnings Growth (yoy) %'] = df2_wide['Quarterly Earnings Growth (yoy) %'].str.strip('%').astype('float') / 100.0
df2_wide['Quarterly Revenue Growth (yoy) %'] = df2_wide['Quarterly Revenue Growth (yoy) %'].str.strip('%').astype('float') / 100.0
df2_wide['Return on Assets (ttm) %'] = df2_wide['Return on Assets (ttm) %'].str.strip('%').astype('float') / 100.0
df2_wide['Return on Equity (ttm) %'] = df2_wide['Return on Equity (ttm) %'].str.strip('%').astype('float') / 100.0
df2_wide['Revenue Per Share (ttm)'] = df2_wide['Revenue Per Share (ttm)'].astype(float)
df2_wide['S&P500 52-Week Change %'] = df2_wide['S&P500 52-Week Change %'].str.strip('%').astype('float') / 100.0
df2_wide['Total Cash Per Share (mrq)'] = df2_wide['Total Cash Per Share (mrq)'].astype(float)
df2_wide['Total Debt/Equity (mrq)'] = df2_wide['Total Debt/Equity (mrq)'].astype(float)
df2_wide['Trailing Annual Dividend Rate'] = df2_wide['Trailing Annual Dividend Rate'].astype(float)
df2_wide['Trailing Annual Dividend Yield %'] = df2_wide['Trailing Annual Dividend Yield %'].str.strip('%').astype('float') / 100.0
df2_wide['200-Day Moving Average'] = df2_wide['200-Day Moving Average'].astype(float)

# Merge the dataframes into company_data
company_data = pd.concat([df1_wide, df2_wide], axis=1, join='inner')

Here we display the dataframe's content so we know what we're working with:

In [72]:
#@title
company_data.head(10)

Attribute,Enterprise Value,Enterprise Value/EBITDA,Enterprise Value/Revenue,Forward P/E,Market Cap (intraday),PEG Ratio (5 yr expected),Price/Book (mrq),Price/Sales (ttm),Trailing P/E,% Held by Insiders,% Held by Institutions,200-Day Moving Average,5 Year Average Dividend Yield %,50-Day Moving Average,52 Week High,52 Week Low,52-Week Change %,Avg Vol (10 day),Avg Vol (3 month),Beta (5Y Monthly),Book Value Per Share (mrq),Current Ratio (mrq),Diluted EPS (ttm),Dividend Date,EBITDA,Ex-Dividend Date,Fiscal Year Ends,Float,Forward Annual Dividend Rate,Forward Annual Dividend Yield %,Gross Profit (ttm),Last Split Date,Last Split Factor,Levered Free Cash Flow (ttm),Most Recent Quarter (mrq),Net Income Avi to Common (ttm),Operating Cash Flow (ttm),Operating Margin (ttm) %,Payout Ratio %,Profit Margin %,Quarterly Earnings Growth (yoy) %,Quarterly Revenue Growth (yoy) %,Return on Assets (ttm) %,Return on Equity (ttm) %,Revenue (ttm),Revenue Per Share (ttm),S&P500 52-Week Change %,Shares Outstanding,"Shares Short (Aug 30, 2020) 4","Shares Short (Sep 14, 2020) 4","Shares Short (prior month Aug 13, 2020) 4","Shares Short (prior month Jul 30, 2020) 4","Short % of Float (Aug 30, 2020) 4","Short % of Float (Sep 14, 2020) 4","Short % of Shares Outstanding (Aug 30, 2020) 4","Short % of Shares Outstanding (Sep 14, 2020) 4","Short Ratio (Aug 30, 2020) 4","Short Ratio (Sep 14, 2020) 4",Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Trailing Annual Dividend Rate,Trailing Annual Dividend Yield %
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,1.87T,22.56,6.83,28.09,1.85T,2.67,25.61,7.01,32.82,0.0007,0.6212,89.49,0.0154,116.96,137.98,53.15,1.0053,183.35M,173.02M,1.28,4.22,1.47,3.30,"Aug 12, 2020",78.67B,"Aug 06, 2020","Sep 27, 2019",17.09B,0.82,0.0076,98.39B,"Aug 30, 2020",4:1,52.26B,"Jun 26, 2020",58.42B,80.01B,0.2452,0.2373,0.2133,0.120,0.109,0.1312,0.6925,273.86B,15.58,0.1081,17.1B,79.68M,NaN,NaN,106.01M,0.47%,NaN,0.47%,NaN,0.39,NaN,93.03B,5.44,122.19B,169.04,0.78,0.0072
KMB,56.58B,13.22,3.00,19.53,49.96B,4.87,186.41,2.67,19.66,0.0034,0.7642,142.22,0.0311,152.93,160.16,110.66,0.0308,1.35M,1.44M,0.48,0.79,0.91,7.45,"Oct 01, 2020",4.34B,"Sep 02, 2020","Dec 30, 2019",339.83M,4.28,0.0292,6.45B,"Nov 02, 2014",1043:1000,2.45B,"Jun 29, 2020",2.56B,4.09B,0.1935,0.5638,0.1358,0.404,0.004,0.1446,9.5523,18.84B,55.10,0.1081,341.05M,NaN,6.07M,5M,NaN,NaN,1.79%,NaN,1.78%,NaN,4.27,1.45B,4.25,8.1B,1636.77,4.20,0.0287
KO,241.70B,18.36,7.04,22.32,209.19B,4.07,11.96,6.13,22.97,0.0070,0.7000,46.76,0.0321,49.14,60.13,36.27,-0.1051,16.19M,15.42M,0.55,4.07,1.09,2.12,"Sep 30, 2020",11.25B,"Sep 13, 2020","Dec 30, 2019",3.86B,1.64,0.0337,22.65B,"Aug 12, 2012",2:1,5.73B,"Jun 25, 2020",9.19B,8.76B,0.2837,0.7642,0.2677,-0.318,-0.285,0.0659,0.4664,34.33B,8.01,0.1081,4.3B,NaN,23.53M,21.57M,NaN,NaN,0.55%,NaN,0.55%,NaN,1.6,19.83B,4.62,52.35B,272.81,1.62,0.0333
MCD,209.48B,22.97,10.96,26.60,160.82B,3.63,NaN,8.53,34.25,0.0012,0.6879,190.19,0.0257,213.56,226.72,124.23,0.0162,2.84M,2.99M,0.68,-12.72,0.91,6.33,"Sep 14, 2020",8.91B,"Aug 30, 2020","Dec 30, 2019",743.5M,5.00,0.0231,11.12B,"Mar 07, 1999",2:1,2.92B,"Jun 29, 2020",4.77B,5.51B,0.3784,0.7781,0.2495,-0.681,-0.305,0.0941,NaN,19.12B,25.54,0.1081,744.1M,NaN,5.49M,6.06M,NaN,NaN,0.74%,NaN,0.74%,NaN,1.81,3.26B,4.38,51.92B,NaN,4.91,0.0227
MSFT,1.45T,21.23,10.16,30.67,1.52T,2.39,12.83,10.78,34.82,0.0142,0.7409,190.15,0.0183,211.43,232.86,132.52,0.4948,35.04M,35.26M,0.89,15.63,2.52,5.76,"Dec 09, 2020",65.26B,"Nov 17, 2020","Jun 29, 2020",7.46B,2.24,0.0110,96.94B,"Feb 17, 2003",2:1,34.26B,"Jun 29, 2020",44.28B,60.67B,0.3703,0.3455,0.3096,-0.151,0.128,0.1126,0.4014,143.02B,18.79,0.1081,7.57B,36.46M,NaN,NaN,36.47M,0.49%,NaN,0.48%,NaN,1.08,NaN,136.49B,18.04,82.11B,69.41,2.04,0.0100
RY,NaN,NaN,NaN,10.92,100.06B,NaN,1.69,2.84,12.07,0.0001,0.4792,66.75,0.0391,73.76,82.74,49.55,-0.1299,535.12k,765.95k,0.71,38.43,NaN,6.29,"Nov 23, 2020",NaN,"Oct 22, 2020","Oct 30, 2019",1.42B,3.28

## Plot basic valuation data to find value investments

In [74]:
#@title
# Plot out the data
chart1 = px.scatter(company_data,
                   x='Quarterly Revenue Growth (yoy) %',
                   y='Price/Sales (ttm)',
                   size = "Trailing P/E",
                   hover_name = company_data.index,
                   title="Quarterly Revenue Growth (yoy) % vs. Price/Sales (ttm) vs. Trailing P/E",
                   width=1250,
                   height=700)

# Update chart layouts
chart1.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    paper_bgcolor="#ffffff",
)

chart1.show()

## Plot dividend data to research dividend stocks

In [66]:
#@title
# Plot out the data
chart1 = px.scatter(company_data,
                   x='Payout Ratio %',
                   y='Trailing Annual Dividend Yield %',
                   size = "5 Year Average Dividend Yield %",
                   hover_name = company_data.index,
                   title="Payout Ratio %' vs. Trailing Annual Dividend Yield % vs. 5 Year Average Dividend Yield %",
                   width=900,
                   height=500)

# Update chart layouts
chart1.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    paper_bgcolor="#eeeeee",
)

chart1.show()

# Retrieve Financial Statements for our Companies

We'll start by pulling the Balance Sheet and then move on to the Income Statement & Cash Flow Statement.

## Balance Sheet

We'll start by getting the Balance Sheet for one company.  That's done like this:

In [67]:
balance_sheet = si.get_balance_sheet("aapl")

display(balance_sheet)

endDate,2019-09-28,2018-09-29,2017-09-30,2016-09-24
Breakdown,,,,
totalLiab,2.480280e+11,2.585780e+11,2.412720e+11,1.934370e+11
totalStockholderEquity,9.048800e+10,1.071470e+11,1.340470e+11,1.282490e+11
otherCurrentLiab,4.324200e+10,3.929300e+10,3.809900e+10,8.243000e+09
totalAssets,3.385160e+11,3.657250e+11,3.753190e+11,3.216860e+11
commonStock,4.517400e+10,4.020100e+10,3.586700e+10,3.125100e+10
otherCurrentAssets,1.235200e+10,1.208700e+10,1.393600e+10,8.283000e+09
retainedEarnings,4.589800e+10,7.040000e+10,9.833000e+10,9.636400e+10
otherLiab,5.050300e+10,4.891400e+10,4.325100e+10,3.900400e+10
treasuryStock,-5.840000e+08,-3.454000e+09,-1.500000e+08,6.340000e+08


But what's more interesting is to get the Balance Sheets for **<u>all</u>** the companies in our list.  That way we can combine them and do interesting things with them.  That's done with the following code:

In [68]:
balance_sheets = {}
for ticker in companies:
    balance_sheets[ticker] = si.get_balance_sheet(ticker)

Now that all the balance sheets are combined, we can look at values from the balance sheets across multiple companies at once.  Since each individual balance sheet may have different column headers (from different dates), we’ll just get the most recent column of data from the balance sheet for each stock.  This code combines the most recent columns from the balance sheets of all companies into one dataframe.

In [69]:
recent_sheets = {ticker : sheet.iloc[:,:1] for ticker,sheet in balance_sheets.items()}

for ticker in recent_sheets.keys():
    recent_sheets[ticker].columns = ["Recent"]

# combine all balance sheets together
combined_sheets = pd.concat(recent_sheets)

# reset index to pull in ticker
combined_sheets = combined_sheets.reset_index()

# update column names
combined_sheets.columns = ["Ticker", "Breakdown", "Recent"]

combined_sheets

,Ticker,Breakdown,Recent
0,AAPL,totalLiab,2.480280e+11
1,AAPL,totalStockholderEquity,9.048800e+10
2,AAPL,otherCurrentLiab,4.324200e+10
3,AAPL,totalAssets,3.385160e+11
4,AAPL,commonStock,4.517400e+10
...,...,...,...
203,RY,netTangibleAssets,6.190600e+10
204,RY,shortTermInvestments,3.449770e+11
205,RY,netReceivables,1.263300e+10
206,RY,longTermDebt,1.063630e+11


From here, we can pull any data from the balance sheets of all the companies. For example, we can look at the Total Assets for each stock like this:

In [75]:
combined_sheets[combined_sheets.Breakdown == "longTermDebt"]

,Ticker,Breakdown,Recent
20,AAPL,longTermDebt,9.180700e+10
48,MSFT,longTermDebt,5.957800e+10
74,WMT,longTermDebt,4.441000e+10
100,TGT,longTermDebt,1.003500e+10
126,MCD,longTermDebt,3.411810e+10
154,KO,longTermDebt,2.753700e+10
179,KMB,longTermDebt,6.242000e+09
206,RY,longTermDebt,1.063630e+11


## Income Statements

Income statements can be downloaded from Yahoo Finance using the get_income_statement method:

In [76]:
income = si.get_income_statement("aapl")

display(income)

endDate,2019-09-28,2018-09-29,2017-09-30,2016-09-24
Breakdown,,,,
researchDevelopment,16217000000,14236000000,11581000000,10045000000
effectOfAccountingCharges,None,None,None,None
incomeBeforeTax,65737000000,72903000000,64089000000,61372000000
minorityInterest,None,None,None,None
netIncome,55256000000,59531000000,48351000000,45687000000
sellingGeneralAdministrative,18245000000,16705000000,15261000000,14194000000
grossProfit,98392000000,101839000000,88186000000,84263000000
ebit,63930000000,70898000000,61344000000,60024000000
operatingIncome,63930000000,70898000000,61344000000,60024000000


Using the income statement, we can examine specific values, such as total revenue, gross profit, total expenses, etc.

To get the **total revenue**, we just need to apply a filter like previously.  Other items like **grossProfit**, **operatingIncome**, etc. works the same.

In [77]:
income.loc["totalRevenue"]

endDate
2019-09-28    260174000000
2018-09-29    265595000000
2017-09-30    229234000000
2016-09-24    215639000000
Name: totalRevenue, dtype: object

## Getting the income statement from each company in our list

Next, let’s retrieve the income statement for each company in our **companies** list.

In [78]:
income_statements = {}
for ticker in companies:
    income_statements[ticker] = si.get_income_statement(ticker)

Now, we can look at metrics in the income statement across multiple companies at once. First, we just need to combine the income statements together, similar to how we combined the balance sheets above.

In [79]:
recent_income_statements = {ticker : sheet.iloc[:,:1] for ticker,sheet in income_statements.items()}

for ticker in recent_income_statements.keys():
    recent_income_statements[ticker].columns = ["Recent"]

combined_income = pd.concat(recent_income_statements)

combined_income = combined_income.reset_index()

combined_income.columns = ["Ticker", "Breakdown", "Recent"]

Now that we have a combined view of the income statements across companies, we can examine specific values in the income statements, such as Total Revenue, for example.

In [80]:
combined_income[combined_income.Breakdown == "totalRevenue"]

,Ticker,Breakdown,Recent
15,AAPL,totalRevenue,260174000000
37,MSFT,totalRevenue,143015000000
59,WMT,totalRevenue,523964000000
81,TGT,totalRevenue,78112000000
103,MCD,totalRevenue,21076500000
125,KO,totalRevenue,37266000000
147,KMB,totalRevenue,18450000000
169,RY,totalRevenue,44138000000


In [81]:
combined_income

,Ticker,Breakdown,Recent
0,AAPL,researchDevelopment,16217000000
1,AAPL,effectOfAccountingCharges,None
2,AAPL,incomeBeforeTax,65737000000
3,AAPL,minorityInterest,None
4,AAPL,netIncome,55256000000
...,...,...,...
171,RY,costOfRevenue,0
172,RY,totalOtherIncomeExpenseNet,-1197000000
173,RY,discontinuedOperations,None
174,RY,netIncomeFromContinuingOps,12871000000


## Cash Flow Statement

In this section, we’ll extract cash flow statements. We can do that using the get_cash_flow method.

In [82]:
flow = si.get_cash_flow("aapl")

Here’s the first few rows of the cash flow statement:

In [83]:
flow.head(20)

endDate,2019-09-28,2018-09-29,2017-09-30,2016-09-24
Breakdown,,,,
investments,58093000000,30845000000,-33542000000,-32022000000
changeToLiabilities,-2548000000,9172000000,8373000000,563000000
totalCashflowsFromInvestingActivities,45896000000,16066000000,-46446000000,-45977000000
netBorrowings,-7819000000,432000000,29014000000,22057000000
totalCashFromFinancingActivities,-90976000000,-87876000000,-17974000000,-20890000000
changeToOperatingActivities,-896000000,30016000000,-8480000000,-902000000
issuanceOfStock,781000000,669000000,555000000,495000000
netIncome,55256000000,59531000000,48351000000,45687000000
changeInCash,24311000000,5624000000,-195000000,-636000000


Now let’s get the cash flow statements of each company on our **companies** list.

In [ ]:
cash_flows = {}
for ticker in companies:
    cash_flows[ticker] = si.get_cash_flow(ticker)

Again, we combine the datasets above, using similar code as before.

In [ ]:
recent_cash_flows = {ticker : flow.iloc[:,:1] for ticker,flow in cash_flows.items()}


for ticker in recent_cash_flows.keys():
    recent_cash_flows[ticker].columns = ["Recent"]


combined_cash_flows = pd.concat(recent_cash_flows)

combined_cash_flows = combined_cash_flows.reset_index()

combined_cash_flows.columns = ["Ticker", "Breakdown", "Recent"]

Now, we can examine information in the cash flow statements across all the stocks in our list.

## Getting dividends paid across companies

One example to look at in a cash flow statement is the amount of dividends paid, which we can see across the companies in our list by using the filter below.

In [ ]:
combined_cash_flows[combined_cash_flows.Breakdown == "dividendsPaid"]

## Getting stock issuance information

Here’s another example – this time, we’ll look at debt-related numbers across the cash flow statements.

In [ ]:
combined_cash_flows[combined_cash_flows.Breakdown == "issuanceOfStock"]

## Reference

Reference Information

In [ ]:
# Print dataframe's columns to a list

print(company_extra_stats_wide.columns.tolist())

In [ ]:
# Pull out just Trailing P/E to another dataframe
trailing_pe = company_stats[company_stats["Attribute"].str.contains("Trailing")]

trailing_pe

In [ ]:
# S&P 500 List
sp500_list = si.tickers_sp500()

print(sp500_list)

In [ ]:
# Dow List
dow_list = si.tickers_dow()

print(dow_list)

In [ ]:
# NASDAQ List
nasdaq_list = si.tickers_nasdaq()

print(nasdaq_list)